In [1]:
import time 

In [144]:
from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader("templates"))
template = env.get_template("prompt1.j2")

prompt_text = template.render(agent_type="Researcher-Agent", role="system")
print(prompt_text)

In [3]:
simulated_database = {
                "mediterranean diet": """The Mediterranean diet is rich in fruits,
                vegetables, whole grains, olive oil, and fish. Studies show it is associated with
                a lower risk of heart disease, improved brain health, and a longer lifespan. Key
                components include monounsaturated fats and antioxidants.""" }



In [101]:
from typing import Union
from pydantic import BaseModel , field_validator , Field 
from typing import Dict, Any , Union , TypedDict
import uuid


class ValidatorContext(BaseModel):
    task: str
    source_summary: str
    draft_post: str



class MCPMessage(BaseModel):
    protocol_version: str = "1.0"
    sender: str= Field(min_length=3)
    content: Union[str, ValidatorContext]
    metadata: Dict[str, Any] = Field(default_factory=dict)


def create_mcp_message(sender: str ,  content: Union[str, ValidatorContext], metadata=None) -> MCPMessage:
    """Create a message in MCP format."""

    metadata = metadata or {} 
    metadata.setdefault("task_id", str(uuid.uuid4()))
    metadata.setdefault("parents",[])

    return MCPMessage(
        sender=sender,
        content=content,
        metadata=metadata   
    )

mcp_anser = create_mcp_message("agent_1", "Hello, this is a test message.", {"priority": "high" , "parents": ["writer-agent"]})
print(mcp_anser.sender)
print(mcp_anser.content )
print(mcp_anser.metadata)

agent_1
Hello, this is a test message.
{'priority': 'high', 'parents': ['writer-agent'], 'task_id': '8069fc3b-cddd-492c-b8d9-2d2f3a4da9cc'}


### robust llm , retries and fallback

In [64]:
from llm_clients import  google_langchain_client  , mimo_v2_flash_client
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage , SystemMessage
from settings import settings

In [65]:
from loguru import logger 
from tenacity import RetryError, retry, stop_after_attempt, wait_exponential, retry_if_exception_type

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=2, max=10),
    retry=retry_if_exception_type(Exception),
    reraise=False
)
def _call_primary(message: list[BaseMessage]) -> AIMessage:
    return google_langchain_client.invoke(message)



def call_llm_robust(system_prompt:str, user_content:str):

    """ calling llm with retries and faallback """ 
    system_prompt= SystemMessage(content=system_prompt)
    user_prompt= HumanMessage(content=user_content)
    message = [system_prompt, user_prompt]

    try:
        logger.info("Invoking primary LLM.")
        return _call_primary(message).content

    ##################### fallback logic #####################
    except Exception as e:
        logger.warning(f"Primary LLM failed after retries: {e}. Switching to fallback.")
    try:
        return mimo_v2_flash_client.invoke(message).content ## fall back
    except Exception:
        return AIMessage(content="system error").content


## researcher agent 



In [66]:
mcp_anser.sender

'agent_1'

In [67]:
def researcher_agent(mcp_input: MCPMessage)-> MCPMessage:
    """ Takes a research topic , finds information and returns summary. """ 

    research_topic= mcp_input.content
    research_result = simulated_database.get(research_topic.lower(), "No Data found")
    
    system_prompt= template.render(agent_type="researcher_agent", role="system")
    summary= call_llm_robust(system_prompt, research_result)
    
    task_id= mcp_input.metadata.get("task_id")
    new_metadata= mcp_input.metadata


    return create_mcp_message(
        sender="researcher_agent",
        content=summary,
        metadata= {"task_id": task_id, "parents":[mcp_input.sender]})

## writer agent 

In [68]:
class MCPMessage(BaseModel):
    protocol_version: str = "1.0"
    sender: str= Field(min_length=3)
    content: str =Field(min_length=3)
    metadata: Dict[str, Any] = Field(default_factory=dict)

In [69]:
def writer_agent(mcp_input:MCPMessage) -> MCPMessage:
    """ This agent takes research findings and writes a shot blog post """ 
    
    research_summary= mcp_input.content
    system_prompt= template.render(agent_type="writer_agent", role="system")
    blog_post= call_llm_robust(system_prompt, research_summary)

    return create_mcp_message(
        sender="writer_agent",
        content=blog_post,
        metadata={"word count": len(blog_post.split()),"task_id": mcp_input.metadata.get("task_id"), "parents":[mcp_input.sender]})

        



In [70]:
mcp_for_researcher= create_mcp_message(
    sender="User",
    content="Mediterranean Diet",)

answer= researcher_agent(mcp_for_researcher)

2026-01-16 14:47:33.431 | INFO     | __main__:call_llm_robust:23 - Invoking primary LLM.


In [71]:
final_answer= writer_agent(answer)

2026-01-16 14:47:35.269 | INFO     | __main__:call_llm_robust:23 - Invoking primary LLM.


In [72]:
final_answer

MCPMessage(protocol_version='1.0', sender='writer_agent', content="This is an excellent summary and a great starting point for a deeper dive into the Mediterranean diet! You've identified the core elements and benefits very effectively.\n\nI appreciate the offer to discuss specific aspects. I'm particularly interested in:\n\n*   **The science behind why these components are beneficial.** I'd love to understand the mechanisms by which monounsaturated fats and antioxidants contribute to the observed health outcomes.\n*   **How to incorporate the Mediterranean diet into your own eating habits.** Practical advice on making this a sustainable lifestyle choice would be very valuable.\n\nLet's start with the science behind the benefits. What are the key ways that monounsaturated fats and antioxidants contribute to improved heart health and brain function, for example?", metadata={'word count': 119, 'task_id': '3a8ea15e-0bf0-4442-a96a-1e9074e34d88', 'parents': ['researcher_agent']})

## Validator Agent

In [161]:
def validator_agent(mcp_input:MCPMessage) -> MCPMessage:
    """ This agent validates the final blog post for quality and coherence """ 

    """ the mcp  inputs is like this sender = sender name ,
     contenxt = {
     "task" : "abcd" ,
     "summary" : "fadsfdasfda"
      "draft_post" : "dfafdada
      }""" 
    ctx = mcp_input.content
    if isinstance(ctx , str):
        return create_mcp_message("validator_agent", "Error : Expected ValidatorContext , got string"   )


    system_prompt= template.render(agent_type="validator_agent", role="system")
    user= template.render(agent_type="validator_agent", role="user", **ctx.model_dump()) # This provides {{ task }}, {{ source_summary }}, and {{ draft_post }} to Jinja

    validation_report= call_llm_robust(system_prompt.strip(), user.strip() )

    return create_mcp_message(
        sender="validator_agent",
        content=validation_report,
        metadata={"task_id": mcp_input.metadata.get("task_id"), "parents":[mcp_input.sender]})

In [162]:
ctx = ValidatorContext(
    task="Validate draft",
    source_summary="Mediterranean diet is healthy.",
    draft_post="The Mediterranean diet includes olive oil, vegetables, and fish."
)

In [163]:
ctx.model_dump()

{'task': 'Validate draft',
 'source_summary': 'Mediterranean diet is healthy.',
 'draft_post': 'The Mediterranean diet includes olive oil, vegetables, and fish.'}

In [164]:
from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader("templates"))
template = env.get_template("prompt1.j2")

prompt_text = template.render(agent_type="Researcher-Agent", role="system")

In [165]:
contex= template.render(agent_type="validator_agent", role="user", **ctx.model_dump())
print(contex.strip())

Context provided below 

Task : Validate draft 


Source Summary  : Mediterranean diet is healthy.  


Draft Post : The Mediterranean diet includes olive oil, vegetables, and fish.


In [166]:
mcp_to_validator = create_mcp_message(
    sender="Orchestrator",
    content=ctx,
    metadata={"task_id": "task_003"}
)

mcp_to_validator.model_dump()


{'protocol_version': '1.0',
 'sender': 'Orchestrator',
 'content': {'task': 'Validate draft',
  'source_summary': 'Mediterranean diet is healthy.',
  'draft_post': 'The Mediterranean diet includes olive oil, vegetables, and fish.'},
 'metadata': {'task_id': 'task_003', 'parents': []}}

In [167]:
ctx.model_dump()

{'task': 'Validate draft',
 'source_summary': 'Mediterranean diet is healthy.',
 'draft_post': 'The Mediterranean diet includes olive oil, vegetables, and fish.'}

In [168]:
validation_answer= validator_agent(mcp_input= mcp_to_validator)

2026-01-16 15:35:12.151 | INFO     | __main__:call_llm_robust:23 - Invoking primary LLM.


In [171]:
validation_answer.content

'pass'

## final Orchestrator

## needs

In [173]:
simulated_database = {
                "mediterranean diet": """The Mediterranean diet is rich in fruits,
                vegetables, whole grains, olive oil, and fish. Studies show it is associated with
                a lower risk of heart disease, improved brain health, and a longer lifespan. Key
                components include monounsaturated fats and antioxidants.""" 
}


from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader("templates"))
template = env.get_template("prompt1.j2")

In [ ]:
def final_orchestrator(initial_goal):
    print("="*50, flush = True)
    print(f"Orchestrator started with goal: {initial_goal}")  
    print("="*50, flush = True)


    research_topic= "mediterranean diet"

    mcp_to_researcher= create_mcp_message(
        sender= "Orchestrator",
        content= research_topic,
    )

    #----------- research agent ----------- 
    mcp_from_researcher= researcher_agent(mcp_to_researcher)

    research_summary= mcp_from_researcher.content


    # ----------------- writer agent ----------------
    mcp_to_writer= create_mcp_message(
        sender="Orchestrator",
        content= mcp_from_researcher.content,)

    
    mcp_from_writer= writer_agent(mcp_to_writer)

    draft_post = mcp_from_writer.content
    #----------- create context -----------
    validation_contex= {"task": research_topic ,
                        "source_summary": research_summary,
                        "draft_post": draft_post} 

    # ---------- validation agent ---------------
    mcp_to_validator= create_mcp_message(
        sender="Orchestrator",
        content=validation_contex)

    mcp_from_validator= validator_agent(mcp_to_validator)
    validation_result= mcp_from_validator.content

    if "pass" in  validation_result.lower():
        return validation_result, mcp_from_validator, validation_contex
        

    else : 
        return  validation_result

In [196]:
user_goal = "Create a blog post about the benefits of the Mediterranean diet."

In [197]:
context_mcp

MCPMessage(protocol_version='1.0', sender='fasdf', content=ValidatorContext(task='Validate draft', source_summary='Mediterranean diet is healthy.', draft_post='The Mediterranean diet includes olive oil, vegetables, and fish.'), metadata={'task_id': '29087254-40a3-4c43-a602-9d064bf3ee6c', 'parents': []})

In [198]:
validation_result,mcp_from_validator, context_mcp = final_orchestrator(user_goal)

Orchestrator started with goal: Create a blog post about the benefits of the Mediterranean diet.


2026-01-16 16:08:49.760 | INFO     | __main__:call_llm_robust:23 - Invoking primary LLM.
2026-01-16 16:08:51.692 | INFO     | __main__:call_llm_robust:23 - Invoking primary LLM.
2026-01-16 16:08:53.141 | INFO     | __main__:call_llm_robust:23 - Invoking primary LLM.
2026-01-16 16:10:42.833 | WARNING  | __main__:call_llm_robust:28 - Primary LLM failed after retries: RetryError[<Future at 0x7fb5b8f0db10 state=finished raised ChatGoogleGenerativeAIError>]. Switching to fallback.


ValueError: too many values to unpack (expected 3)

In [ ]:
validation_result

MCPMessage(protocol_version='1.0', sender='validator_agent', content='pass', metadata={'task_id': 'task_003', 'parents': ['Orchestrator']})

In [ ]:
validation_result

## Unlock a Healthier You: The Mediterranean Diet Secret!

Dreaming of a vibrant, long life? The answer might be on your plate! The Mediterranean diet is a delicious way to boost your well-being, focusing on fresh, flavorful foods like fruits, veggies, whole grains, and heart-healthy olive oil.

This eating pattern is packed with goodness, especially monounsaturated fats and powerful antioxidants. Research shows it's a superstar for your heart, helping to lower disease risk and keep your blood pressure in check. Plus, it's a brain booster, supporting cognitive function and potentially warding off age-related decline.

Ready to embrace this flavorful lifestyle? Start by adding more colorful produce, swapping butter for olive oil, and enjoying fish a couple of times a week. Your body will thank you!


In [193]:
mcp_from_validator

MCPMessage(protocol_version='1.0', sender='validator_agent', content='pass', metadata={'task_id': '7c9d6802-370a-4170-bf39-bfed83ac5ed7', 'parents': ['Orchestrator']})